<a href="https://colab.research.google.com/github/Zen-Teng/Stylos/blob/main/FineTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Unsloth

In [4]:
!pip install "unsloth @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-y3e5fggz/unsloth_b2a19724f15a4297ad92ed67bf2fa0e4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-y3e5fggz/unsloth_b2a19724f15a4297ad92ed67bf2fa0e4
  Resolved https://github.com/unslothai/unsloth.git to commit a50b4337ae526744711f54f9aed6e82c778280fd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2026.2.1-py3-none-any.whl size=430126 sha256=924940b99018b19c7b54c8d7c9789ccb153026a18115220dddf420d031a58e71
  Stored in directory: /tmp/pip-ephem-wheel-cache-kou1qbx5/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth


In [2]:
#### Load Existing model using Unsloth and give one example from Mergers and Acquistions News

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(model_name = "unsloth/llama-3-8b-bnb-4bit", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


mna_news_input = """HCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022."""
mna_news_instruction = """Which company is the Acquirer. """


ModuleNotFoundError: No module named 'unsloth'

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
inputs = tokenizer([alpaca_prompt.format(mna_news_instruction, mna_news_input, "")], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 90)
tokenizer.batch_decode(outputs)[0]

'<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhich company is the Acquirer. \n\n### Input:\nHCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022.\n\n### Response:\nHCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022.\n<|end_of_text|>'